# Đồ án môn học Lập trình song song ứng dụng

# Phát hiện khuôn mặt bằng Cascaded-Classifiers

## Giáo viên: Trần Trung Kiên

## Sinh viên:
- 1712628 - Huỳnh Hiếu Nhân
- 1712657 - Chung Kiệt Phong
- 1712869 - Đào Duy Tuấn

---

# 1. Mô tả ứng dụng

- Nhóm em chọn ứng dụng Phát hiện khuôn mặt bằng Cascaded-Classifiers
	- Input: một tấm ảnh RGB
	- Output: tấm ảnh đầu vào có bounding box ở các khuôn mặt trong ảnh (nếu có)
	- Ý nghĩa của ứng dụng trong thực tế:
		- Hỗ trợ tính năng autofocus trên máy ảnh
		- Là bước quan trọng trong các hệ thống sử dụng nhận diện khuôn mặt: giám sát, an ninh, cơ sở dữ liệu, chăm sóc khách hàng ...
		- Ứng dụng để tự động đề xuất gắn thẻ cho ảnh trên mạng xã hội
- Lý do cần tăng tốc ứng dụng:
	- Có nhiều ý nghĩa thực tế
	- Ứng dụng sẽ chạy chậm nếu cài đặt tuần tự
	- Nhu cầu xử lý dữ liệu lớn, ảnh có độ phân giải cao, video, streaming video trong thời gian thực
- Tiềm năng song song hóa của ứng dụng: 
  - Các khung cửa sổ để phát hiện khuôn mặt ở mọi vị trí trong ảnh và các kích thước khác nhau của khung cửa số xử lý độc lập với nhau
  - Tính tuần tự của các giai đoạn kiểm tra có thách thức lớn nhưng tốc độ xử lý được gia tăng đáng kể nếu có thể song song hóa phần này

---

# 2. Cài đặt tuần tự

## 2.1. Thiết kế

- Input: một tấm ảnh RGB

- (các bước)

- ...

- Output: tấm ảnh đầu vào có bounding box ở các khuôn mặt trong ảnh (nếu có)

---